<a href="https://colab.research.google.com/github/hyunaeee/KU-SW-Academy/blob/main/1013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Information Retrieval => Text Mining      != NLP
                         ;비정형데이터(텍스트)    ;말의 순서(Sequence)
=> 정보요구(Information needs)로부터 가장 연관된(Relevance) 문서 집합을 순위화(Ranking)

[Concept]
1. 정보요구=Query, representation
2. 데이터(문서) representation
3. 1-2의 관계를 잘 연관성(Relevance) => Search Engine / Retrieval Model
1.2. 둘다, BoW(Independent each term)+N-Gram => Improved BoW

[Components]
1. Crawler; 데이터수집기
2. Indexer; 텍스트전처리
   ; Structure => Term-Doc. Matrix => Iverted Index(역색인)
                  Doc.-Term Matrix
                        term1 term2 ...
                  doc1  => 책 제일 뒤 색인표
                  doc2  (D,V)
                  ...
                  O(|Q|*D*|D|)
                        doc1 doc2 ...
                  term1 (V,D)
                  term2
                  ...
                  O(|Q|*|L|)
   ; TDM 구조 변환 => Linked List
     List(순서;DataStructure) != Array(같은Type, 연속된)
3. Analyzer => Representation; 데이터베이스
4. Model
--------------------------------------
5. Evaluation(User Relevance Feedback)
6. Personalized...
import re
from os import listdir

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))
# 1. Matrix - 어제 (D*V*bits)
# 2. Dictionary (D*|L|)
# 3. List (tuple)
# 4. InvertedIndex(File) (tuple(32bits*3))
from nltk.tokenize import regexp_tokenize

# 2. Dictionary
# TDM = [
#       0:Term1:  [Doc1, Doc2, ...] |D|,
#         []
#       ]
# TDM = {
#         {Term1:[Doc1, Doc2, ...]} |t1ㅌD|
#       }

p1 = re.compile(r'\s+')

D = list()
V = list()
TDM = dict()

for file in fileids('naver'):
    i = len(D)
    D.append(file)
    with open(file, 'r', encoding='utf8') as fp:
        for t in regexp_tokenize(p1.sub(' ', fp.read()), r'\b\w+\b'):
            for g in ngram(t):
                if g not in V:
                    V.append(g)
                j = V.index(g)

                if j not in TDM.keys():
                    TDM[j] = list()

                if i not in TDM[j]:
                    TDM[j].append(i)
len(TDM), len(V), len(D), len(TDM[0])
(13241, 13241, 87, 1)
sum([len(v) for k,v in TDM.items()])/len(V) # => |L|
2.7234347858923043
i2w = lambda i:V[i]
w2i = lambda w:V.index(w)

i2d = lambda i:D[i]
d2i = lambda d:D.index(d)
query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'

result = list()

for t in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(t): # ngram => tokenizing
        if g in V:
            j = w2i(g)
            result.append(TDM[j])
# prefix, infix, postfix
#           1 + 2
#   + 1 2          1 2 +

# OR
# candidates = list()
# for r in result:
#     candidates.extend(r)
# list(set(candidates))

# AND
candidates = list()
for r in result:
    candidates = list(set(candidates).intersection(r))
candidates
[]
from nltk.tokenize import regexp_tokenize

# 3. List
# TDM = {
#         {Term1:[Doc1, Doc2, ...]} |t1ㅌD|
#       }

# TDM = {
#         {Term1:Posting 몇 번째 위치}
#       }
# Posting = [
#            0:(doc1, 1, next:1)
#            1:(doc2, 1, next:-1)
#           ]

p1 = re.compile(r'\s+')

D = list()
V = list()
TDM = dict()
Posting = list()

for file in fileids('naver'):
    i = len(D)
    D.append(file)
    with open(file, 'r', encoding='utf8') as fp:
        for t in regexp_tokenize(p1.sub(' ', fp.read()), r'\b\w+\b'):
            for g in ngram(t):
                if g not in V:
                    V.append(g)
                j = V.index(g)

                if j not in TDM.keys():
                    pos = len(Posting)
                    Posting.append((i,1,-1))
                    TDM[j] = pos
                else:
                    pos = len(Posting)
                    Posting.append((i,1,TDM[j]))
                    TDM[j] = pos
query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'

result = list()

for t in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(t): # ngram => tokenizing
        if g in V:
            result.append(list())

            j = w2i(g)
            pos = TDM[j]
            while pos != -1:
                i, freq, npos = Posting[pos]
                pos = npos

                result[-1].append(i)
            result[-1] = list(set(result[-1]))
sorted(result[0])
[22, 28, 34, 43, 53, 72, 79, 81, 82, 84]
# Linked List => Posting
# on-disk
fp = open()
0->fp.tell()
data
    ^
    위치:fp.tell()


1주소:(다음주소:파일위치)
        3주소:****
   4주소:(다음주소:-1)
            2주소:(다음주소:3)

[0:1, 1:2, ...] => List

class {
    int no;
    void* next;
} NODE;

class {
    private public protcted NODE* head;
    NODE* tail;

    int (*push(void*));
    int (*pop(void*));
} LinkedList;

LinkedList list;
list->head = (NODE*)malloc(sizeof(NODE));
memset(list->head, 0, sizeof(NODE));

List().append()
list.push = push;

int push(void* a) {
    (NODE*)a->
}

list.push()
from struct import pack, unpack
these can be preceded by a decimal repeat count:
      x: pad byte (no data); c:char; b:signed byte; B:unsigned byte;
      ?: _Bool (requires C99; if not available, char is used instead)
      h:short; H:unsigned short; i:int; I:unsigned int;
      l:long; L:unsigned long; f:float; d:double; e:half-float.
    Special cases (preceding decimal count indicates length):
      s:string (array of char); p: pascal string (with count byte).
    Special cases (only available in native format):
      n:ssize_t; N:size_t;
      P:an integer type that is wide enough to hold a pointer.
    Special case (not in native mode unless 'long long' in platform C):
      q:long long; Q:unsigned long long
unpack('iii', pack('iii', 1, 1, -1))
(1, 1, -1)
from nltk.tokenize import regexp_tokenize

# 4. File TDM
# TDM = {
#         {Term1:Posting 몇 번째 위치}
#       }
# Posting = [
#            0:(doc1, 1, next:0)
#            1:(doc2, 1, next:-1)
#           ]
# FILE
# Posting = 0:(123:fp)3:(110:fp)21-1

p1 = re.compile(r'\s+')

D = list()
V = list()
TDM = dict()

Posting = open('posting.dat', 'wb')

for file in fileids('naver'):
    i = len(D)
    D.append(file)
    with open(file, 'r', encoding='utf8') as fp:
        for t in regexp_tokenize(p1.sub(' ', fp.read()), r'\b\w+\b'):
            for g in ngram(t):
                if g not in V:
                    V.append(g)
                j = V.index(g)

                if j not in TDM.keys():
                    pos = Posting.tell()
                    Posting.write(pack('iii', i, 1, -1))
                    TDM[j] = pos
                else:
                    pos = Posting.tell()
                    Posting.write(pack('iii', i, 1, TDM[j]))
                    TDM[j] = pos

Posting.close()
query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'

result = list()

Posting = open('posting.dat', 'rb')

for t in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(t): # ngram => tokenizing
        if g in V:
            result.append(list())

            j = w2i(g)
            pos = TDM[j]
            while pos != -1:
                Posting.seek(pos)
                i, freq, npos = unpack('iii', Posting.read(12))
                pos = npos

                result[-1].append(i)
            result[-1] = list(set(result[-1]))

Posting.close()
sorted(result[0])
[22, 28, 34, 43, 53, 72, 79, 81, 82, 84]
unpack('d', pack('d', 0.0))
(0.0,)
from nltk.tokenize import regexp_tokenize

# 4. File TDM - 빈도를 고려할 수 있도록
# 개별 코퍼스 별로 처리

p1 = re.compile(r'\s+')

D = list()
V = list()
Dictionary = dict()

# w => 새로생성 = 0
Posting = open('posting.dat', 'wb')
Posting.close()

# 전체 코퍼스(data)
for file in fileids('naver'):
    i = len(D)
    D.append(file)

    # Local
    # 코퍼스 1개 open
    with open(file, 'r', encoding='utf8') as fp:
        corpus = fp.read()

    localTDM = dict()
    localPosting = open('local.dat', 'wb')

    # Tokenizing+Normalizing
    for t in regexp_tokenize(p1.sub(' ', corpus), r'\b\w+\b'):
        for g in ngram(t):
            # 문서 1개 작업하는 중에, 단어가 처음 등장
            if g not in localTDM.keys():
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, -1))
                localTDM[g] = pos
            else:
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, localTDM[g]))
                localTDM[g] = pos

    localPosting.close()

    # Update; Local -> Global
    Posting = open('posting.dat', 'ab')
    localPosting = open('local.dat', 'rb')

    # k=단어, v=파일위치
    for k, v in localTDM.items():
        if k not in V:
            V.append(k)

        j = V.index(k)

        freq = 0
        pos = v
        while pos != -1:
            localPosting.seek(pos)
            f, npos = unpack('ii', localPosting.read(8))
            pos = npos
            freq += 1

        if j not in Dictionary.keys():
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, -1))
            Dictionary[j] = pos
        else:
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, Dictionary[j]))
            Dictionary[j] = pos

    localPosting.close()
    Posting.close()
# Local
LocalTDM = {'단어':'로컬파일 위치', ...}
LocalPosting
1다음위치1다음위치1다음위치1다음위치
while -1:
    freq += 1

# Global
Dictionary = {j:글로벌파일위치, ...}
Posting
ifreq다음위치ifreq다음위치ifreq다음위치...
(i, freq, 다음위치)
fp = open('posting.dat', 'wb') # a-appending 마지막위치
print(fp.tell())
fp.close()
0
query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'

result = list()

Posting = open('posting.dat', 'rb')

for t in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(t): # ngram => tokenizing
        if g in V:
            result.append(list())

            j = w2i(g)
            pos = Dictionary[j]
            while pos != -1:
                Posting.seek(pos)
                i, freq, npos = unpack('iii', Posting.read(12))
                pos = npos

                result[-1].append(i)
            result[-1] = list(set(result[-1]))

Posting.close()
P(Y|X,theta=p)

P(Y) = P(Yㅌresult|X) + P(!Yㅌresult|X)

(Java)Luciene <= Elastic Search(IR, Database X)
                 => Inverted Index(Database X)
                 Hadoop => Database X